# 3D Material Bead Packing XCT Simulation Instance Segmentation

In [2]:
import os

import itk
import numpy as np
from matplotlib import pyplot as plt
from itkwidgets import view, label_statistics
import itkwidgets

In [3]:
file_name = '../data/bead_pack.tif'
beads = itk.imread(file_name)
viewer = view(beads, ui_collapsed=True)
viewer

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageUC3; pr…

In [4]:
sharpened = itk.unsharp_mask_image_filter(beads, sigma=2.0, threshold=10.0, amount=5.0)
viewer.image = sharpened
viewer.mode = 'x'

In [5]:
invert = itk.invert_intensity_image_filter(sharpened)
thresholded = itk.otsu_threshold_image_filter(invert)
viewer.image = thresholded
viewer.interpolation = False

In [14]:
type(watershed)

itk.itkImagePython.itkImageSS3

In [7]:
StructuringElementType = itk.FlatStructuringElement[3]
structuring_element = StructuringElementType.Ball(3, True)
opened = itk.binary_morphological_opening_image_filter(thresholded,
                                                       kernel=structuring_element)
viewer.image = opened

In [9]:
filled = itk.binary_fillhole_image_filter(opened, fully_connected=True)
viewer.image = filled

In [10]:
distance = itk.signed_maurer_distance_map_image_filter(filled, inside_is_positive=False)
view(distance)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [11]:
watershed = itk.morphological_watershed_image_filter(distance, level=7.0)

In [12]:
filled_cast = itk.cast_image_filter(filled, ttype=(type(filled), type(watershed)))
watershed_mask = itk.mask_image_filter(watershed, filled_cast)

In [12]:
names = [(0, 'Background'), (1, 'Bead')]
view(beads, label_image=watershed_mask, 
      label_image_names=names,
      label_image_blend=0.7,
      rotate=True,
      shadow=False,
      gradient_opacity=0.5)

Viewer(geometries=[], gradient_opacity=0.5, interpolation=False, label_image_blend=0.7, label_image_names=[(0,…

In [13]:
label_statistics(beads, watershed_mask, histogram=True)

AppLayout(children=(Viewer(interpolation=False, label_image_blend=0.75, layout=Layout(grid_area='header'), ren…